In [2]:
import msgpack
import cv2
# import easyocr
import imagehash
import os
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd

In [3]:
# list files in downloaded_images folder
card_files = set(os.listdir("downloaded_images"))
print(len(card_files))

17121


In [4]:
data_source = 'cards.msgpack'
with open(data_source, 'rb') as file:
    data = msgpack.load(file, raw=False)

In [5]:
def get_image_route(card_id: str):
    return card_id + '.png'

In [12]:
def get_hashes(img):
    perceptual = imagehash.phash(img, 32, 8)
    difference = imagehash.dhash(img, 32)
    wavelet = imagehash.whash(img, 32)
    color = imagehash.colorhash(img)
    return {
        'perceptual': perceptual,
        'difference': difference,
        'wavelet': wavelet,
        'color': color
    }

In [13]:
hashes = []

for card in data:
    file_name = get_image_route(card['id'])
    img_src = os.path.join('downloaded_images', file_name)
    if file_name not in card_files:
        continue

    img = Image.open(img_src)

    card_hashes = get_hashes(img)
    card_hashes['id'] = card['id']
    hashes.append(card_hashes)

df = pd.DataFrame(hashes)

In [14]:
df.head()

,perceptual,difference,wavelet,color,id
0,9eaebefa6ebc147ac9057145994bfe6924d11bc566d357...,b9c644ca4bd871374cdec23716d6a6cd16f8028716e089...,7ffffffea57ffd93800ffb03c066f005c13fe001c07ffc...,06e00000000,hgss4-1
1,bffa80571faaa3578542af55f147efc449055d607aa15c...,b59804e42ce6e1d39a67621317bcdea317f65e031dcc77...,7ffffffece37ffc3ef01ff03c0dffffbc03ffffb807f13...,02580000000,xy5-1
2,dabe115a5abcd542c10eeaf9a55b2efb1c5bbab51ffb0c...,7d20032913d2061b12d68a1b2ce1822629872017930761...,fffff1ff8009e709817fffcb8000c003c000f003c80030...,02400010000,pl1-1
3,9abb80535bba8455e100fba6e144bf8ec447eaf38cf680...,3d2003a913db061b12db8c4b2ce6cd272133d21783d3d0...,fffff9ffc009ff0983ffffc380004003c0002803c000f8...,03601008000,dp3-1
4,bbffa0ff3bbc017fc007d150c44fff50e401ff503d81ef...,bf8003ec23431f9338333f1316e3606717a64887134998...,7ffffffefd0003f3fc000783813db003c3f1f803c1a028...,023c0000000,det1-1


In [15]:
df.to_csv('card_hashes_32b.csv')

In [17]:
df.to_pickle('card_hashes_32b.pickle')

In [16]:
img = cv2.imread(r'.\downloaded_images\pl1-SH4.png')
img.shape

(825, 600, 3)

In [35]:
df['perceptual'][0] - df['perceptual'][1]

464